In [1]:
import pandas as pd
import numpy as np
import pathlib
import requests
import json

In [53]:
data_path = pathlib.Path.cwd()/'notices_of_repatriation.csv'
notices_of_repatriation = pd.read_csv(data_path)
print(notices_of_repatriation)
#notices_of_repatriation['Link']

    Publication Date                                              Title  \
0         12/20/2019             San Diego Museum of Man, San Diego, CA   
1         11/27/2019  Thomas Burke Memorial Washington State Museum,...   
2          11/8/2019  University of Tennessee, Department of Anthrop...   
3          10/9/2019  Sam Noble Oklahoma Museum of Natural History, ...   
4          10/9/2019  Colgate University, Longyear Museum of Anthrop...   
..               ...                                                ...   
810         2/2/1995         Museum of New Mexico, Santa Fe, New Mexico   
811       12/21/1994                          Heard Museum, Phoenix, AZ   
812       11/16/1994          Arizona State Capitol Museum, Phoenix, AZ   
813         7/7/1994  Anchorage Museum of History and Art, Anchorage...   
814        5/31/1994                  Nebraska State Historical Society   

                                                  Link  
0    https://www.federalregister.gov/docum

In [54]:
url = 'https://www.nps.gov/common/uploads/sortable_dataset/nagpra/FC3C107D-A964-AEF6-2DFE0CA7DC186182/FC3C107D-A964-AEF6-2DFE0CA7DC186182.json'
response = requests.get(url)
repatriation_dict = json.loads(response.content)
#print(repatriation_dict)
nps_repatriation = pd.DataFrame(data= \
                               json.loads(response.content)['DATA'], \
                               columns=json.loads(response.content)['COLUMNS'])
nps_repatriation





,Publication Date,Title,Link
0,12/20/2019,"San Diego Museum of Man, San Diego, CA",https://www.federalregister.gov/documents/2019...
1,11/27/2019,"Thomas Burke Memorial Washington State Museum,...",https://www.federalregister.gov/documents/2019...
2,11/8/2019,"University of Tennessee, Department of Anthrop...",https://www.federalregister.gov/documents/2019...
3,10/9/2019,"Sam Noble Oklahoma Museum of Natural History, ...",https://www.federalregister.gov/documents/2019...
4,10/9/2019,"Colgate University, Longyear Museum of Anthrop...",https://www.federalregister.gov/documents/2019...
...,...,...,...
810,2/2/1995,"Museum of New Mexico, Santa Fe, New Mexico",https://www.federalregister.gov/documents/1995...
811,12/21/1994,"Heard Museum, Phoenix, AZ",https://www.federalregister.gov/documents/1994...
812,11/16/1994,"Arizona State Capitol Museum, Phoenix, AZ",https://www.federalregister.gov/documents/1994...
813,7/7/1994,"Anchorage Museum of History and Art, Anchorage...",https://www.federalregister.gov/documents/1994...


In [55]:
test_url = nps_repatriation.loc[0]['Link']
#test_response=requests.get(test_url)
#test_response.content

In [56]:
'''
Using magic command to load python script
(Doing it this way because I had trouble with importing & passing arguments)

Need to talk to DE re separating the script.
Renamed script from Download_JSON.py because that was no longer descriptive
'''

# %load ../urlgetter.py
from bs4 import BeautifulSoup
#import sys
#html_page_source=sys.argv[1]

def get_urls (html):
    '''
    Parses HTML to return URLs of interest
    (i.e., links to pages with JSON and with full text)
    '''
    soup = BeautifulSoup(html, "html.parser")
    fulltext_url = None
    json_url = None
    for link in soup.find_all('a'):
        if link.string == None:
            continue
        if fulltext_url != None and json_url != None:
            break
        if link.string == 'JSON: Normalized attributes and metadata':
            json_url = link.get('href')
            
        '''
        could refine logic here. "text" covers 
        "XML: Original full text XML" 
        as well as "basic text format"
        '''
        if 'text' in link.string.lower(): 
            fulltext_url = link.get('href')
    return json_url, fulltext_url

In [62]:
'''
Adding and filling 'json_url' and 'fulltext_url' columns to nps_inventories

Takes a long time to run. There probably exists a more efficient way than .apply
Only need to run it once, though.
'''

def fill_urls(row):
    repatriate_url = row['Link']
    test_response=requests.get(repatriate_url)
    html_page_source = str(test_response.content)[2:-1]
    #%run -i Download_JSON
    json_url, fulltext_url = get_urls(html_page_source)
    row['json_url'] = json_url
    row['fulltext_url'] = fulltext_url
    print(row.name, json_url, fulltext_url)
    return row

nps_repatriation['json_url'] = np.nan
nps_repatriation['fulltext_url'] = np.nan


nps_repatriation = nps_repatriation.apply(fill_urls, axis=1)

0 https://www.federalregister.gov/api/v1/documents/2019-27489?publication_date=2019-12-20 https://www.federalregister.gov/documents/full_text/xml/2019/12/20/2019-27489.xml
0 https://www.federalregister.gov/api/v1/documents/2019-27489?publication_date=2019-12-20 https://www.federalregister.gov/documents/full_text/xml/2019/12/20/2019-27489.xml
1 https://www.federalregister.gov/api/v1/documents/2019-25728?publication_date=2019-11-27 https://www.federalregister.gov/documents/full_text/xml/2019/11/27/2019-25728.xml
2 https://www.federalregister.gov/api/v1/documents/2019-24409?publication_date=2019-11-08 https://www.federalregister.gov/documents/full_text/xml/2019/11/08/2019-24409.xml
3 https://www.federalregister.gov/api/v1/documents/2019-22045?publication_date=2019-10-09 https://www.federalregister.gov/documents/full_text/xml/2019/10/09/2019-22045.xml
4 https://www.federalregister.gov/api/v1/documents/2019-22049?publication_date=2019-10-09 https://www.federalregister.gov/documents/full_tex

47 https://www.federalregister.gov/api/v1/documents/2018-10781?publication_date=2018-05-21 https://www.federalregister.gov/documents/full_text/xml/2018/05/21/2018-10781.xml
48 https://www.federalregister.gov/api/v1/documents/2018-07701?publication_date=2018-04-13 https://www.federalregister.gov/documents/full_text/xml/2018/04/13/2018-07701.xml
49 https://www.federalregister.gov/api/v1/documents/2018-07703?publication_date=2018-04-13 https://www.federalregister.gov/documents/full_text/xml/2018/04/13/2018-07703.xml
50 https://www.federalregister.gov/api/v1/documents/2018-06836?publication_date=2018-04-04 https://www.federalregister.gov/documents/full_text/xml/2018/04/04/2018-06836.xml
51 https://www.federalregister.gov/api/v1/documents/2018-06834?publication_date=2018-04-04 https://www.federalregister.gov/documents/full_text/xml/2018/04/04/2018-06834.xml
52 https://www.federalregister.gov/api/v1/documents/2018-03633?publication_date=2018-02-22 https://www.federalregister.gov/documents/fu

95 https://www.federalregister.gov/api/v1/documents/2017-08880?publication_date=2017-05-03 https://www.federalregister.gov/documents/full_text/xml/2017/05/03/2017-08880.xml
96 https://www.federalregister.gov/api/v1/documents/2017-05094?publication_date=2017-03-15 https://www.federalregister.gov/documents/full_text/xml/2017/03/15/2017-05094.xml
97 https://www.federalregister.gov/api/v1/documents/2017-05093?publication_date=2017-03-15 https://www.federalregister.gov/documents/full_text/xml/2017/03/15/2017-05093.xml
98 https://www.federalregister.gov/api/v1/documents/2017-04855?publication_date=2017-03-13 https://www.federalregister.gov/documents/full_text/xml/2017/03/13/2017-04855.xml
99 https://www.federalregister.gov/api/v1/documents/2017-04405?publication_date=2017-03-07 https://www.federalregister.gov/documents/full_text/xml/2017/03/07/2017-04405.xml
100 https://www.federalregister.gov/api/v1/documents/2017-04404?publication_date=2017-03-07 https://www.federalregister.gov/documents/f

143 https://www.federalregister.gov/api/v1/documents/2015-29360?publication_date=2015-11-17 https://www.federalregister.gov/documents/full_text/xml/2015/11/17/2015-29360.xml
144 https://www.federalregister.gov/api/v1/documents/2015-29355?publication_date=2015-11-17 https://www.federalregister.gov/documents/full_text/xml/2015/11/17/2015-29355.xml
145 https://www.federalregister.gov/api/v1/documents/2015-26296?publication_date=2015-10-15 https://www.federalregister.gov/documents/full_text/xml/2015/10/15/2015-26296.xml
146 https://www.federalregister.gov/api/v1/documents/2015-26289?publication_date=2015-10-15 https://www.federalregister.gov/documents/full_text/xml/2015/10/15/2015-26289.xml
147 https://www.federalregister.gov/api/v1/documents/2015-21501?publication_date=2015-08-31 https://www.federalregister.gov/documents/full_text/xml/2015/08/31/2015-21501.xml
148 https://www.federalregister.gov/api/v1/documents/2015-21499?publication_date=2015-08-31 https://www.federalregister.gov/docume

191 https://www.federalregister.gov/api/v1/documents/2014-20222?publication_date=2014-08-26 https://www.federalregister.gov/documents/full_text/xml/2014/08/26/2014-20222.xml
192 https://www.federalregister.gov/api/v1/documents/2014-17735?publication_date=2014-07-28 https://www.federalregister.gov/documents/full_text/xml/2014/07/28/2014-17735.xml
193 https://www.federalregister.gov/api/v1/documents/2014-17734?publication_date=2014-07-28 https://www.federalregister.gov/documents/full_text/xml/2014/07/28/2014-17734.xml
194 https://www.federalregister.gov/api/v1/documents/2014-14748?publication_date=2014-06-24 https://www.federalregister.gov/documents/full_text/xml/2014/06/24/2014-14748.xml
195 https://www.federalregister.gov/api/v1/documents/2014-14746?publication_date=2014-06-24 https://www.federalregister.gov/documents/full_text/xml/2014/06/24/2014-14746.xml
196 https://www.federalregister.gov/api/v1/documents/2014-14744?publication_date=2014-06-24 https://www.federalregister.gov/docume

...

579 https://www.federalregister.gov/api/v1/documents/04-1885?publication_date=2004-01-29 https://www.federalregister.gov/documents/full_text/xml/2004/01/29/04-1885.xml
580 https://www.federalregister.gov/api/v1/documents/04-143?publication_date=2004-01-06 https://www.federalregister.gov/documents/full_text/xml/2004/01/06/04-143.xml
581 https://www.federalregister.gov/api/v1/documents/04-142?publication_date=2004-01-06 https://www.federalregister.gov/documents/full_text/xml/2004/01/06/04-142.xml
582 https://www.federalregister.gov/api/v1/documents/03-30567?publication_date=2003-12-10 https://www.federalregister.gov/documents/full_text/xml/2003/12/10/03-30567.xml
583 https://www.federalregister.gov/api/v1/documents/03-30569?publication_date=2003-12-10 https://www.federalregister.gov/documents/full_text/xml/2003/12/10/03-30569.xml
584 https://www.federalregister.gov/api/v1/documents/03-30565?publication_date=2003-12-10 https://www.federalregister.gov/documents/full_text/xml/2003/12/10/03-

628 https://www.federalregister.gov/api/v1/documents/02-21994?publication_date=2002-08-29 https://www.federalregister.gov/documents/full_text/xml/2002/08/29/02-21994.xml
629 https://www.federalregister.gov/api/v1/documents/02-21997?publication_date=2002-08-29 https://www.federalregister.gov/documents/full_text/xml/2002/08/29/02-21997.xml
630 https://www.federalregister.gov/api/v1/documents/02-17429?publication_date=2002-07-11 https://www.federalregister.gov/documents/full_text/xml/2002/07/11/02-17429.xml
631 https://www.federalregister.gov/api/v1/documents/02-17085?publication_date=2002-07-11 https://www.federalregister.gov/documents/full_text/xml/2002/07/11/02-17085.xml
632 https://www.federalregister.gov/api/v1/documents/02-17416?publication_date=2002-07-11 https://www.federalregister.gov/documents/full_text/xml/2002/07/11/02-17416.xml
633 https://www.federalregister.gov/api/v1/documents/02-17414?publication_date=2002-07-11 https://www.federalregister.gov/documents/full_text/xml/2002

677 https://www.federalregister.gov/api/v1/documents/01-7437?publication_date=2001-03-26 https://www.federalregister.gov/documents/full_text/xml/2001/03/26/01-7437.xml
678 https://www.federalregister.gov/api/v1/documents/01-6841?publication_date=2001-03-20 https://www.federalregister.gov/documents/full_text/xml/2001/03/20/01-6841.xml
679 https://www.federalregister.gov/api/v1/documents/01-6897?publication_date=2001-03-20 https://www.federalregister.gov/documents/full_text/xml/2001/03/20/01-6897.xml
680 https://www.federalregister.gov/api/v1/documents/01-6842?publication_date=2001-03-20 https://www.federalregister.gov/documents/full_text/xml/2001/03/20/01-6842.xml
681 https://www.federalregister.gov/api/v1/documents/01-5941?publication_date=2001-03-09 https://www.federalregister.gov/documents/full_text/xml/2001/03/09/01-5941.xml
682 https://www.federalregister.gov/api/v1/documents/01-5938?publication_date=2001-03-09 https://www.federalregister.gov/documents/full_text/xml/2001/03/09/01-5

726 https://www.federalregister.gov/api/v1/documents/99-31363?publication_date=1999-12-03 https://www.govinfo.gov/content/pkg/FR-1999-12-03/html/99-31363.htm
727 https://www.federalregister.gov/api/v1/documents/99-28290?publication_date=1999-11-01 https://www.govinfo.gov/content/pkg/FR-1999-11-01/html/99-28290.htm
728 https://www.federalregister.gov/api/v1/documents/99-27125?publication_date=1999-10-18 https://www.govinfo.gov/content/pkg/FR-1999-10-18/html/99-27125.htm
729 https://www.federalregister.gov/api/v1/documents/99-27127?publication_date=1999-10-18 https://www.govinfo.gov/content/pkg/FR-1999-10-18/html/99-27127.htm
730 https://www.federalregister.gov/api/v1/documents/99-27128?publication_date=1999-10-18 https://www.govinfo.gov/content/pkg/FR-1999-10-18/html/99-27128.htm
731 https://www.federalregister.gov/api/v1/documents/99-24856?publication_date=1999-09-24 https://www.govinfo.gov/content/pkg/FR-1999-09-24/html/99-24856.htm
732 https://www.federalregister.gov/api/v1/documents

778 https://www.federalregister.gov/api/v1/documents/98-5407?publication_date=1998-03-03 https://www.govinfo.gov/content/pkg/FR-1998-03-03/html/98-5407.htm
779 https://www.federalregister.gov/api/v1/documents/98-5406?publication_date=1998-03-03 https://www.govinfo.gov/content/pkg/FR-1998-03-03/html/98-5406.htm
780 https://www.federalregister.gov/api/v1/documents/98-4685?publication_date=1998-02-24 https://www.govinfo.gov/content/pkg/FR-1998-02-24/html/98-4685.htm
781 https://www.federalregister.gov/api/v1/documents/98-2458?publication_date=1998-02-02 https://www.govinfo.gov/content/pkg/FR-1998-02-02/html/98-2458.htm
782 https://www.federalregister.gov/api/v1/documents/98-660?publication_date=1998-01-12 https://www.govinfo.gov/content/pkg/FR-1998-01-12/html/98-660.htm
783 https://www.federalregister.gov/api/v1/documents/98-659?publication_date=1998-01-12 https://www.govinfo.gov/content/pkg/FR-1998-01-12/html/98-659.htm
784 https://www.federalregister.gov/api/v1/documents/97-33292?public

In [63]:
print(len(nps_repatriation.json_url.unique())==len(nps_repatriation))
print(len(nps_repatriation.fulltext_url.unique())==len(nps_repatriation))

True
True


In [64]:
nps_repatriation.to_csv('repatriation_urls.csv', index=False)
repatriation_urls = pd.read_csv('repatriation_urls.csv')

In [65]:
repatriation_urls

,Publication Date,Title,Link,json_url,fulltext_url
0,12/20/2019,"San Diego Museum of Man, San Diego, CA",https://www.federalregister.gov/documents/2019...,https://www.federalregister.gov/api/v1/documen...,https://www.federalregister.gov/documents/full...
1,11/27/2019,"Thomas Burke Memorial Washington State Museum,...",https://www.federalregister.gov/documents/2019...,https://www.federalregister.gov/api/v1/documen...,https://www.federalregister.gov/documents/full...
2,11/8/2019,"University of Tennessee, Department of Anthrop...",https://www.federalregister.gov/documents/2019...,https://www.federalregister.gov/api/v1/documen...,https://www.federalregister.gov/documents/full...
3,10/9/2019,"Sam Noble Oklahoma Museum of Natural History, ...",https://www.federalregister.gov/documents/2019...,https://www.federalregister.gov/api/v1/documen...,https://www.federalregister.gov/documents/full...
4,10/9/2019,"Colgate University, Longyear Museum of Anthrop...",https://www.federalregister.gov/documents/2019...,https://www.federalregister.gov/api/v1/documen...,https://www.federalregister.gov/documents/full...
...,...,...,...,...,...
810,2/2/1995,"Museum of New Mexico, Santa Fe, New Mexico",https://www.federalregister.gov/documents/1995...,https://www.federalregister.gov/api/v1/documen...,https://www.govinfo.gov/content/pkg/FR-1995-02...
811,12/21/1994,"Heard Museum, Phoenix, AZ",https://www.federalregister.gov/documents/1994...,https://www.federalregister.gov/api/v1/documen...,https://www.govinfo.gov/content/pkg/FR-1994-12...
812,11/16/1994,"Arizona State Capitol Museum, Phoenix, AZ",https://www.federalregister.gov/documents/1994...,https://www.federalregister.gov/api/v1/documen...,https://www.govinfo.gov/content/pkg/FR-1994-11...
813,7/7/1994,"Anchorage Museum of History and Art, Anchorage...",https://www.federalregister.gov/documents/1994...,https://www.federalregister.gov/api/v1/documen...,https://www.govinfo.gov/content/pkg/FR-1994-07...
